In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

## Language modeling

### Data

In [3]:
PATH='data/wikitext-2/'
%ls {PATH}

wiki.test.tokens*  wiki.train.tokens*  wiki.valid.tokens*


In [4]:
!head -5 {PATH}wiki.train.tokens

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcomers . Char

In [5]:
!wc -lwc {PATH}wiki.train.tokens

   36718  2051910 10797148 data/wikitext-2/wiki.train.tokens


In [6]:
!wc -lwc {PATH}wiki.valid.tokens

   3760  213886 1121681 data/wikitext-2/wiki.valid.tokens


In [9]:
TEXT = data.Field(lower=True)
FILES = dict(train='wiki.train.tokens', validation='wiki.valid.tokens', test='wiki.test.tokens')
bs,bptt = 80,70
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(365, 12980, 1, 2051911)

In [10]:
#md.trn_ds[0].text[:12], next(iter(md.trn_dl))

### Train

In [15]:
emb_sz = 200
n_hid = 500
n_layers = 3
learner = md.get_model(opt_fn=SGD_Momentum(0.7), emb_sz=emb_sz, n_hid=n_hid, n_layers=n_layers)
reg_fn=partial(seq2seq_reg, alpha=2, beta=1)

In [19]:
clip=0.3
lr=10
cycles=1
weight_decay=1e-6
learner.reg_fn=reg_fn
learner.clip=clip
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay)

epoch      trn_loss   val_loss                              
    0      6.162434   5.815104  



[5.8151037890505854]

In [21]:
cycles=6
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                              
    0      5.919782   5.604272  
    1      5.781447   5.44062                               
    2      5.700526   5.379003                              
    3      5.645984   5.293015                              
    4      5.533223   5.183098                              
    5      5.460034   5.121386                              
    6      5.446831   5.108704                              
    7      5.427698   5.070461                              
    8      5.354173   5.009482                              
    9      5.272665   4.935861                              
    10     5.22003    4.889684                              
    11     5.193845   4.865676                              
    12     5.159422   4.839201                              
    13     5.149208   4.829961                              
    14     5.148971   4.828529                              
    15     5.177523   4.848187                      

[4.407092474346701]

In [22]:
learner.save('lm_420')

In [ ]:
learner.load('lm_420')
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save('lm_419')

In [ ]:
learner.load('lm_419')
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save('lm_418')

In [ ]:
learner.load('lm_418')
math.exp(4.17)

### Test

In [24]:
m=learner.model
s=[""". <eos> The game began development in 2010 , carrying over a large portion of the work 
done on Valkyria Chronicles II . While it retained the standard features of """.split()]
t=TEXT.numericalize(s)

m[0].bs=1
m.reset()
res,*_ = m(t)

In [25]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['the', '<unk>', 'a', '"', 'an', 'its', 'this', 'his', 'their', 'all']

In [29]:
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))

the 

/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  after removing the cwd from sys.path.
/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  """


RuntimeError: input must have 3 dimensions, got 2

### End